In [ ]:
from pathlib import Path
from mistralai import Mistral
import os

from common.keys import MISTRAL_API_KEY_PAYANTE
from common.path import COMMERCIAL_ONE_DRIVE_PATH, rapatrie_file


CHIEN_QUI_FUME_PATH = (
    COMMERCIAL_ONE_DRIVE_PATH
    / "2 - DOSSIERS à l'ETUDE"
    / "CHIEN QUI FUME (Le) - 75001 PARIS - 33 Rue du PONT-NEUF"
)

# pdf_path = Path(r"C:\Users\lvolat\Desktop\Echeancier def 240507 MAISON MULOT SAS.pdf")
pdf_path = list(CHIEN_QUI_FUME_PATH.glob("*SOCIAL*/*salaires_Annuel_2024*"))[0]
pdf_file_path = rapatrie_file(pdf_path)

cocoAI - INFO - Journal_des_salaires_Annuel_2024.pdf already exists in 5._SOCIAL


In [10]:
def process_pdf_by_Mistral(pdf_path):
    client = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)
    uploaded_pdf = client.files.upload(
        file={
            "file_name": pdf_file_path.name,
            "content": open(pdf_file_path, "rb"),
        },
        purpose="ocr"
    )
    client.files.retrieve(file_id=uploaded_pdf.id)  
    signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)
    
    client2 = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)
    ocr_response = client2.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": signed_url.url,
        }
    )
    return ocr_response

In [12]:
import json

# pdf_path = Path(r"C:\Users\lvolat\Desktop\Echeancier def 240507 MAISON MULOT SAS.pdf")
SOCIAL = list(CHIEN_QUI_FUME_PATH.glob("*SOCIAL*"))[0]
PAIE = SOCIAL / "PAIE 10"
pdf_path = list(PAIE.glob("*Djiedji*"))[0]
pdf_file_path = rapatrie_file(pdf_path)
ocr_response = process_pdf_by_Mistral(pdf_file_path)
response_dict = json.loads(ocr_response.model_dump_json())
print(json.dumps(response_dict, indent=4))

cocoAI - INFO - C:\Users\lvolat\COMPTOIRS ET COMMERCES\COMMERCIAL - Documents\2 - DOSSIERS à l'ETUDE\CHIEN QUI FUME (Le) - 75001 PARIS - 33 Rue du PONT-NEUF\5. SOCIAL\PAIE 10\GALLA_202410 CAMARA Djiedji_Normal.pdf has been copied to C:\Users\lvolat\Documents\cocoAI\data\2_-_DOSSIERS_lETUDE\CHIEN_QUI_FUME_Le_-_75001_PARIS_-_33_Rue_du_PONT-NEUF\5._SOCIAL\PAIE_10


{
    "pages": [
        {
            "index": 0,
            "markdown": "# SAS GALLA AU CHIEN QUI FUME \n\n33 Rue du Pont Neuf\n75001 PARIS\n\n## SIRET 31013032300028 APE 5610A\n\nURSSAF ILE-DE-France No 117000001500171157\n\n## N ${ }^{\\circ}$ de S\u00e9curit\u00e9 Sociale 173089933511655\n\nMatricule 6\nD\u00e9partement\nService\nCoefficient\nEchelon 2\nEmploi Barman\nStatut Employ\u00e9\nEntr\u00e9e 15/07/2011\nCommentaire\n\n## Salaire au service : 3603.09\u20ac\n\n| $\\mathrm{N}^{\\circ}$ | D\u00e9signation | Nombre | Base | Part salariale |  | Part Patronale |  |\n| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |\n|  |  |  |  | Taux | Montant | Taux | Montant |\n| 0100 | Salaire de base | 151.67 |  | 13.260 | 2011.14 |  |  |\n| 0106 | Compl\u00e9ment de Service |  |  |  | 1328.22 |  |  |\n| 0200 | Heures supp. structurelles \u00e0 $110 \\%$ Hcr (pour calcul all\u00e9g.) | 17.33 |  | 15.218 | 263.73 |  |  |\n| 3205 | Avantages en nature repas non exo. Hcr | 21.00 |  | 

In [ ]:
from IPython.display import Markdown, display
display(Markdown(response_dict['pages'][0]['markdown']))

In [17]:
response_dict['pages'][0]['markdown']

"# SAS GALLA AU CHIEN QUI FUME \n\n33 Rue du Pont Neuf\n75001 PARIS\n\n## SIRET 31013032300028 APE 5610A\n\nURSSAF ILE-DE-France No 117000001500171157\n\n## N ${ }^{\\circ}$ de Sécurité Sociale 173089933511655\n\nMatricule 6\nDépartement\nService\nCoefficient\nEchelon 2\nEmploi Barman\nStatut Employé\nEntrée 15/07/2011\nCommentaire\n\n## Salaire au service : 3603.09€\n\n| $\\mathrm{N}^{\\circ}$ | Désignation | Nombre | Base | Part salariale |  | Part Patronale |  |\n| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |\n|  |  |  |  | Taux | Montant | Taux | Montant |\n| 0100 | Salaire de base | 151.67 |  | 13.260 | 2011.14 |  |  |\n| 0106 | Complément de Service |  |  |  | 1328.22 |  |  |\n| 0200 | Heures supp. structurelles à $110 \\%$ Hcr (pour calcul allég.) | 17.33 |  | 15.218 | 263.73 |  |  |\n| 3205 | Avantages en nature repas non exo. Hcr | 21.00 |  | 4.150 | 87.15 |  |  |\n| 3999 | Total du brut | 169.00 |  |  | 3690.24 |  |  |\n| 4000 | Vieillesse déplafonnée |  | 3690.24 

In [97]:
from mistralai import DocumentURLChunk, Mistral, TextChunk
client = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)

# Get OCR results for processing
ocr_markdown = response_dict["pages"][0]["markdown"]

# Get structured response from model
chat_response = client.chat.complete(
    model="mistral-large-latest",
    messages=[
        {
            "role": "user",
            "content": [
                TextChunk(
                    text=(
                        f"Voici le bulletin de salaire en markdown:\n\n{ocr_markdown}\n.\n"
                        "Extrais toutes ces informations au format pandas"
                    )
                ),
            ],
        }
    ],
    response_format={"type": "json_object"},
    temperature=0,
)


In [102]:

# Parse and return JSON response
Markdown(ocr_markdown)


# SAS GALLA AU CHIEN QUI FUME 

33 Rue du Pont Neuf
75001 PARIS

## SIRET 31013032300028 APE 5610A

URSSAF ILE-DE-France No 117000001500171157

## N ${ }^{\circ}$ de Sécurité Sociale 173089933511655

Matricule 6
Département
Service
Coefficient
Echelon 2
Emploi Barman
Statut Employé
Entrée 15/07/2011
Commentaire

## Salaire au service : 3603.09€

| $\mathrm{N}^{\circ}$ | Désignation | Nombre | Base | Part salariale |  | Part Patronale |  |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|  |  |  |  | Taux | Montant | Taux | Montant |
| 0100 | Salaire de base | 151.67 |  | 13.260 | 2011.14 |  |  |
| 0106 | Complément de Service |  |  |  | 1328.22 |  |  |
| 0200 | Heures supp. structurelles à $110 \%$ Hcr (pour calcul allég.) | 17.33 |  | 15.218 | 263.73 |  |  |
| 3205 | Avantages en nature repas non exo. Hcr | 21.00 |  | 4.150 | 87.15 |  |  |
| 3999 | Total du brut | 169.00 |  |  | 3690.24 |  |  |
| 4000 | Vieillesse déplafonnée |  | 3690.24 | 0.400 | $-14.76$ | 2.020 | $-74.54$ |
| 4010 | Vieillesse plafonnée |  | 3690.24 | 6.900 | $-254.63$ | 8.550 | $-315.52$ |
| 4050 | FNAL plafonné |  | 3690.24 |  |  | 0.100 | $-3.69$ |
| 4060 | Allocations familiales |  | 3690.24 |  |  | 3.450 | $-127.31$ |
| 4080 | Maladie |  | 3690.24 |  |  | 7.000 | $-258.32$ |
| 4100 | Accident du travail |  | 3690.24 |  |  | 1.840 | $-67.90$ |
| 4120 | Solidarité autonomie |  | 3690.24 |  |  | 0.300 | $-11.07$ |
| 4350 | Forfait social à $8 \%$ (prévoyance) |  | 40.60 |  |  | 8.000 | 8.000 |
| 4420 | Réduction salariale cotisations heures supp. |  |  |  | 29.83 |  |  |
| 4430 | Déduction forfaitaire patronale heures supp. | 17.33 |  |  |  | 0.500 | 8.66 |
| 4500 | CSG déductlble |  | 3407.15 | 6.800 | $-231.69$ |  |  |
| 4600 | Versement mobilité |  | 3690.24 |  |  | 3.200 | $-118.09$ |
| 4740 | Contribution au dialogue social |  | 3690.24 |  |  | 0.016 | $-0.59$ |
| 4800 | Assurance chômage TA |  | 3690.24 |  |  | 4.050 | $-149.45$ |
| 4815 | Assurance chômage modulation |  | 3690.24 |  |  | 0.490 | $-18.08$ |
| 4900 | AGS TA + TB |  | 3690.24 |  |  | 0.250 | $-9.23$ |
| 6000 | KLESIA (ex-Groupe D ET O) Retraite T1 |  | 3690.24 | 3.150 | $-116.24$ | 4.720 | $-174.18$ |
| 6008 | KLESIA (ex-Groupe D ET O) Retraite CEG T1 |  | 3690.24 | 0.860 | $-31.74$ | 1.290 | $-47.60$ |
| 7000 | KLESIA P Prévoyance décée |  | 3690.24 | 0.066 | $-2.44$ | 0.224 | $-8.27$ |
| 7010 | KLESIA P Prévoyance incapacité |  | 3690.24 | 0.176 | $-6.49$ | 0.044 | $-1.62$ |
| 7050 | KLESIA P Prévoyance invalidité |  | 3690.24 | 0.152 | $-5.61$ | 0.038 | $-1.40$ |
| 7100 | KLESIA P Prévoyance rente éducation |  | 3690.24 | 0.036 | $-1.33$ | 0.124 | $-4.58$ |
| 7601 | KLESIA P Mutuelle obl. sur Plafond |  | 3864.00 | 0.640 | $-24.73$ | 0.640 | $-24.73$ |
| 7999 | Total imposable |  |  |  | 2791.41 |  |  |
| 8000 | CSG/CRDS non déductible |  | 3407.15 | 2.900 | $-98.81$ |  |  |
| 8020 | CSG non déductible sur sommes non imposables (hors épargne |  | 259.11 | 9.700 | $-25.13$ |  |  |
| 8531 | Navigo mensuel |  |  |  | 43.20 |  |  |
| 8999 | Total dû |  |  |  | 2949.67 |  |  |
| 9200 | Acompte |  |  |  | $-600.00$ |  |  |
| 9450 | Reprise avantages en nature |  |  |  | 87.15 |  |  |
| 9500 | Formation professionnelle |  | 3690.24 |  |  | 1.000 | $-36.90$ |
| 9600 | Taxe d'apprentissage (part principale) |  | 3690.24 |  |  | 0.590 | $-21.77$ |
| 9602 | Taxe d'apprentissage (solide) |  | 3690.24 |  |  | 0.090 | $-3.32$ |
| 9987 | Montant net social |  | 2906.47 |  |  |  |  |
| 9995 | Total net à payer avant prélèvement à la source |  |  |  | 2262.52 |  |  |
| 9996 | Evolution rémunération taux de cotisations 2018 |  |  |  | 53.92 |  |  |
| 9997 | Prélèvement à la source (taux personnalisé) |  |  | 2791.41 | 4.300 | 120.03 |  |  |
| 9999 | Total net à payer |  |  |  | 2142.49 |  |  |


|  | CONGES PAYES |  | RTT |  | CHARGES |  |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|  | N-1 | N | N-1 | N |  |  |
| Acquis | 30.00 | 12.50 |  |  |  |  |
| Pris | 13.50 |  |  | Totales | 2256.52 |  |
| Solde | 16.50 | 12.50 |  | Coût global | 5206.19 |  |
|  | Heures | Brut | Base CP | Charges salariales | Charges patronales | Plafond | Assiette plafonnée | Net imposable | Montant PAS | Mt. HC/HS défis. |
| Mois | 169.00 | 3690.24 | 3690.24 | 783.77 | 1472.75 | 3864.00 | 3690.24 | 2791.41 | 120.03 | 246.11 |
| Cumul | 1690.00 | 34430.85 | 18219.44 | 7304.15 | 13852.60 | 38640.00 | 34430.85 | 25858.90 | 1353.73 | 2507.54 |

In [103]:
import re

def replace_figures(text):
    # Regular expression pattern to match figures in the form $number$
    pattern = re.compile(r'\$([0-9]+(\.[0-9]+)?)\$')

    # Replace the matched pattern with just the number
    replaced_text = pattern.sub(r'\1', text)

    return replaced_text

# Example usage
text = "Here are some figures: $15.2$, $30$, and $45.75$. Let's replace them."
result = replace_figures(text)
print(result)


Here are some figures: 15.2, 30, and 45.75. Let's replace them.


In [78]:
import re
import pandas as pd
from io import StringIO
import json

my_str='''| Some Title | Some Description             | Some Number |
|------------|------------------------------|-------------|
| Dark Souls | This is a fun game           | 5           |
| Bloodborne | This one is even better      | 2           |
| Sekiro     | This one is also pretty good | 110101      |'''

def mrkd2json(inp):
    lines = inp.split('\n')
    ret=[]
    keys=[]
    for i,l in enumerate(lines):
        if i==0:
            keys=[_i.strip() for _i in l.split('|')]
        elif i==1: continue
        else:
            ret.append({keys[_i]:v.strip() for _i,v in enumerate(l.split('|')) if  _i>0 and _i<len(keys)-1})
    return json.dumps(ret, indent = 4) 
# print(mrkd2json(my_str))



def extract_tables(markdown_string):
    # Regular expression pattern to match Markdown tables
    table_pattern = re.compile(r'(\|.*?\|\n\|[-|: ]*\|\n(\|.*?\|\n?)+)', re.MULTILINE | re.DOTALL)

    # Find all matches of the table pattern in the input string
    tables = table_pattern.findall(markdown_string)

    # Extract and return the tables
    return [table[0] for table in tables]

def tables_to_dataframes(tables):
    dataframes = []
    for table in tables:
        # Split the table into lines
        lines = table.strip().split('\n')

        # Extract the header and alignment lines
        header = lines[0]
        alignment = lines[1]

        # Determine column alignment based on the alignment line
        alignments = []
        for item in alignment.split('|')[1:-1]:
            item = item.strip()
            if item.startswith(':') and item.endswith(':'):
                alignments.append('center')
            elif item.startswith(':'):
                alignments.append('left')
            elif item.endswith(':'):
                alignments.append('right')
            else:
                alignments.append('left')  # Default to left alignment

        # Convert the table string to a DataFrame
        df = pd.read_csv(StringIO('\n'.join(lines)), sep='|', header=0, index_col=False, skipinitialspace=True)

        # Drop the first and last empty columns
        df = df.iloc[:, 1:-1]

        # Apply the alignment to the DataFrame (for display purposes)
        for col, align in zip(df.columns, alignments):
            df[col] = df[col].astype(str).str.strip()

        # Save the DataFrame to a CSV file for intermediate inspection
        csv_file = f"intermediate_table_{len(dataframes) + 1}.csv"
        df.to_csv(csv_file, index=False)
        print(f"Intermediate CSV saved as {csv_file}")

        dataframes.append(df)

    return dataframes

# Example usage
markdown_string = """
# Sample Markdown

Here is a table:

| Header 1 | Header 2 |
|:---------|:--------:|
| Row 1    |  Data 1  |
| Row 2    |  Data 2  |

And another table:

| Name    | Age |
|:--------|----:|
| Alice   |  30 |
| Bob     |  25 |
"""

tables = extract_tables(markdown_string)
print(tables)
# print([mrkd2json(t) for t in tables])

# dataframes = tables_to_dataframes(tables)

# for i, df in enumerate(dataframes):
#     print(f"DataFrame {i+1}:\n{df}\n")


['| Header 1 | Header 2 |\n|:---------|:--------:|\n| Row 1    |', '|\n| Row 2    |  Data 2  |\n\nAnd another table:\n\n| Name    | Age |\n|:--------|----:|\n| Alice   |']


In [94]:
import re

def extract_markdown_tables(text):
    # Regular expression pattern to match Markdown tables with optional alignment
    table_pattern = re.compile(r'(\|.*\|\n\|[-:| ]+\|\n(\|.*\|\n?)+)', re.MULTILINE)

    # Find all matches of the pattern in the text
    tables = table_pattern.findall(text)

    # Extract the full table strings from the matches
    extracted_tables = [match[0] for match in tables]

    return extracted_tables

# Example usage
markdown_text = """
Here is a sample Markdown text with tables:

| Name   | Age |
|:------:|:---:|
| Alice  | 30  |
| Bob    | 25  |

And another table with centered text:

| Fruit  | Color |
|:------:|:-----:|
| Apple  | Red   |
| Banana | Yellow|
"""

tables = extract_markdown_tables(markdown_text)
for i, table in enumerate(tables):
    print(f"Table {i+1}:\n{table}")


Table 1:
| Name   | Age |
|:------:|:---:|
| Alice  | 30  |
| Bob    | 25  |

Table 2:
| Fruit  | Color |
|:------:|:-----:|
| Apple  | Red   |
| Banana | Yellow|



In [ ]:
import json

# my_str='''| Header 1 | Header 2 |
# |:---------|:--------:|
# | Row 1    | Data 1   |
# | Row 2    | Data 2   |'''

def mrkd2df(inp):
    lines = inp.split('\n')
    ret=[]
    keys=[]
    for i,l in enumerate(lines):
        if i==0:
            keys=[_i.strip() for _i in l.split('|')]
        elif i==1: continue
        else:
            ret.append({keys[_i]:v.strip() for _i,v in enumerate(l.split('|')) if  _i>0 and _i<len(keys)-1})
    # return pd.DataFrame.from_dict(json.dumps(ret, indent = 4)) 
    return pd.DataFrame.from_dict(ret)


ocr_markdown = replace_figures(ocr_markdown)
tables = extract_markdown_tables(ocr_markdown)
df = mrkd2df(tables[0])

# print(df.dropna())


In [101]:
df

,$\mathrm{N}^{\circ}$,Désignation,Nombre,Base,Part salariale,,Part Patronale
0,,,,,Taux,Montant,Taux
1,0100,Salaire de base,151.67,,13.260,,
2,0106,Complément de Service,,,,,
3,0200,Heures supp. structurelles à $110 \%$ Hcr (pou...,17.33,,15.218,,
4,3205,Avantages en nature repas non exo. Hcr,21.00,,4.150,,
5,3999,Total du brut,169.00,,,,
6,4000,Vieillesse déplafonnée,,3690.24,0.400,$-74.54$,2.020
7,4010,Vieillesse plafonnée,,3690.24,6.900,$-315.52$,8.550
8,4050,FNAL plafonné,,3690.24,,$-3.69$,0.100
9,4060,Allocations familiales,,3690.24,,$-127.31$,3.450


[
    {
        "Some Title": "Dark Souls",
        "Some Description": "This is a fun game",
        "Some Number": "5"
    },
    {
        "Some Title": "Bloodborne",
        "Some Description": "This one is even better",
        "Some Number": "2"
    },
    {
        "Some Title": "Sekiro",
        "Some Description": "This one is also pretty good",
        "Some Number": "110101"
    }
]


In [ ]:
tables

In [ ]:
chat_response.choices[0].message.content


In [ ]:
response_dict2= json.loads(chat_response.choices[0].message.content)
with open('salaires.json','w') as f:
    f.write(json.dumps(response_dict2))

In [ ]:
import pandas as pd

response_dict2 = json.loads(chat_response.choices[0].message.content)
with open("salaires.json", "w", encoding="utf-8") as f:
    print(Path("salaires.json").resolve())
    f.write(json.dumps(response_dict2, ensure_ascii=False))

df = pd.DataFrame.from_dict(
    response_dict2["SAS GALLA AU CHIEN QUI FUME"]["Détails du salaire"],
    orient='index'
)
df

C:\Users\lvolat\Documents\cocoAI\cocoAI\salaires.json


,Désignation,Nombre,Base,Part salariale,Montant,Part Patronale
0100,Salaire de base,151.67,,13.260,,
0106,Complément de Service,,,,,
0200,Heures supp. structurelles à $110 %$ Hcr (pour...,17.33,,15.218,,
3205,Avantages en nature repas non exo. Hcr,21.00,,4.150,,
3999,Total du brut,169.00,,,,
4000,Vieillesse déplafonnée,,3690.24,0.400,-74.54,2.020
4010,Vieillesse plafonnée,,3690.24,6.900,-315.52,8.550
4050,FNAL plafonné,,3690.24,,-3.69,0.100
4060,Allocations familiales,,3690.24,,-127.31,3.450
4080,Maladie,,3690.24,,-258.32,7.000


In [41]:
df

,0100,0106,0200,3205,3999,4000,4010,4050,4060,4080,...,9200,9450,9500,9600,9602,9987,9995,9996,9997,9999
Désignation,Salaire de base,Complément de Service,Heures supp. structurelles à $110 %$ Hcr (pour...,Avantages en nature repas non exo. Hcr,Total du brut,Vieillesse déplafonnée,Vieillesse plafonnée,FNAL plafonné,Allocations familiales,Maladie,...,Acompte,Reprise avantages en nature,Formation professionnelle,Taxe d'apprentissage (part principale),Taxe d'apprentissage (solide),Montant net social,Total net à payer avant prélèvement à la source,Évolution rémunération taux de cotisations 2018,Prélèvement à la source (taux personnalisé),Total net à payer
Nombre,151.67,,17.33,21.00,169.00,,,,,,...,,,,,,,,,,
Base,,,,,,3690.24,3690.24,3690.24,3690.24,3690.24,...,,,3690.24,3690.24,3690.24,2906.47,,,,
Part salariale,13.260,,15.218,4.150,,0.400,6.900,,,,...,,,,,,,,,2791.41,
Montant,,,,,,-74.54,-315.52,-3.69,-127.31,-258.32,...,,,-36.90,-21.77,-3.32,,,,,
Part Patronale,,,,,,2.020,8.550,0.100,3.450,7.000,...,,,1.000,0.590,0.090,,,,120.03,


In [49]:
response_dict2["SAS GALLA AU CHIEN QUI FUME"]["Détails du salaire"]

{'0100': {'Désignation': 'Salaire de base',
  'Nombre': '151.67',
  'Base': '',
  'Part salariale': '13.260',
  'Montant': '',
  'Part Patronale': ''},
 '0106': {'Désignation': 'Complément de Service',
  'Nombre': '',
  'Base': '',
  'Part salariale': '',
  'Montant': '',
  'Part Patronale': ''},
 '0200': {'Désignation': 'Heures supp. structurelles à $110 %$ Hcr (pour calcul allég.)',
  'Nombre': '17.33',
  'Base': '',
  'Part salariale': '15.218',
  'Montant': '',
  'Part Patronale': ''},
 '3205': {'Désignation': 'Avantages en nature repas non exo. Hcr',
  'Nombre': '21.00',
  'Base': '',
  'Part salariale': '4.150',
  'Montant': '',
  'Part Patronale': ''},
 '3999': {'Désignation': 'Total du brut',
  'Nombre': '169.00',
  'Base': '',
  'Part salariale': '',
  'Montant': '',
  'Part Patronale': ''},
 '4000': {'Désignation': 'Vieillesse déplafonnée',
  'Nombre': '',
  'Base': '3690.24',
  'Part salariale': '0.400',
  'Montant': '-74.54',
  'Part Patronale': '2.020'},
 '4010': {'Désigna

In [ ]:
#j essaie le modele mistral small 

import os
from mistralai import Mistral

model = "mistral-small-latest"

# Initialize the Mistral client
client = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)

# Define the messages for the chat
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Quelle est la ligne de compte 7762?"
            },
            {
                "type": "document_url",
                "document_url": signed_url.url,
            }
        ]
    }
]

# Get the chat response
chat_response = client.chat.complete(
    model=model,
    messages=messages
)

# Print the content of the response
print(chat_response.choices[0].message.content)


La ligne de compte 7762 n'est pas directement visible dans les extraits de documents fournis. Cependant, en général, les lignes de compte en comptabilité peuvent être associées à diverses catégories de dépenses ou de revenus. Voici quelques étapes pour identifier ou comprendre ce que pourrait être la ligne de compte 7762 :

1. **Consulter le Plan Comptable Général (PCG)** : En France, le PCG est un référentiel qui classe les comptes comptables. La ligne 7762 pourrait être une sous-catégorie de charges de personnel ou d'autres types de dépenses.

2. **Vérifier les Documents Comptables** : Si vous avez accès à un plan comptable détaillé ou à un manuel de comptabilité de l'entreprise, vous pourriez y trouver la définition précise de la ligne 7762.

3. **Demander à un Expert-Comptable** : Un expert-comptable ou un responsable financier pourrait rapidement identifier la ligne 7762 en consultant les documents comptables de l'entreprise.

4. **Rechercher dans les Logiciels de Comptabilité** :